In [ ]:
!git clone https://github.com/shannanyinxiang/SPTS.git

In [ ]:
!conda init bash
!conda create -n spts python=3.7 -y

In [ ]:
!conda env list

In [ ]:
!source /opt/conda/bin/activate spts

In [ ]:
!conda install ipykernel -y

In [ ]:
!pip install gdown
import gdown
import os

In [ ]:
%cd SPTS

In [ ]:
print(os.getcwd())

In [ ]:
env_path = "/opt/conda/envs/spts"
PYTHON = os.path.join(env_path, "bin", "python")
# PYTHON = "/kaggle/working/SPTS"

In [ ]:
!{PYTHON} -m pip install torch==1.8.1 torchvision==0.9.1 torchaudio==0.8.1
# !conda install pytorch==1.8.1 torchvision==0.9.1 torchaudio==0.8.1 -c pytorch

In [ ]:
requirements = open("requirements.txt", "w")
requirements.write('''bezier
opencv-python
Pillow==8.2.0
pycocotools
tqdm
editdistance
shapely''')
requirements.close()
!{PYTHON} -m pip install -r requirements.txt

In [ ]:
url = 'https://drive.google.com/uc?id=1HuhbtL_pp-Qj2T7dLzKPgv7ypTQjgxRM'
gdown.download(url, 'ic15.pth', quiet=False)

In [ ]:
# url = 'https://drive.google.com/uc?id=12XFwSlZvNQ-_I16bSWgbnmD7-jMqR-CW'
# gdown.download(url, 'ke2ne5.pth', quiet=False)

In [ ]:
!mkdir data

In [ ]:
# %cd data
# %mkdir totaltext_val
# %cd ..

In [ ]:
# !ls ./data/icdar2015

In [ ]:
%cd data
!gdown 1THhzo_WH1RY5DlGdBfjRA_dwu9tAmQUE
!unzip icdar2015.zip
%cd ..

In [ ]:
# !{PYTHON} main.py --eval --data_root ./data/ --val_dataset ic15_val --output_folder ./ --resume ./ic15.pth --tfm_pre_norm --visualize

In [ ]:
# !ls ./results/ep349/vis

In [ ]:
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg

In [ ]:
# image_path = "./results/ep349/vis/img_9.jpg"

# img = mpimg.imread(image_path)

# fig = plt.figure(figsize=(10, 10))

# print(f"Image shape: {img.shape}")

# plt.imshow(img)
# plt.axis('off')

# plt.show()

In [ ]:
lr_scheduler = open('./optim/lr_scheduler.py', 'w')
lr_scheduler.write('''import warnings
from torch.optim.lr_scheduler import _LRScheduler

class LinearDecayLR(_LRScheduler):
    def __init__(self, optimizer, args, last_epoch=-1, verbose=False):
        if args.finetune:
            self.lrs = [args.lr] * (args.epochs + 1)
        else:
            warmup_lr = [args.warmup_min_lr + ((args.lr - args.warmup_min_lr) * i / args.warmup_epochs) for i in range(args.warmup_epochs)]
            decay_lr = [max(i * args.lr / args.epochs, args.min_lr) for i in range(args.epochs - args.warmup_epochs)]
            decay_lr.reverse()
            self.lrs = warmup_lr + decay_lr + decay_lr[-1:]
        
        self.lr_backbone_ratio = args.lr_backbone_ratio
        
        for param_group in optimizer.param_groups:
            param_group['initial_lr'] = 0.00001
        
        
        super(LinearDecayLR, self).__init__(optimizer, last_epoch, verbose)
    
    def get_lr(self):
        if not self._get_lr_called_within_step:
            warnings.warn("To get the last learning rate computed by the scheduler, "
                        "please use `get_last_lr()`.", UserWarning) 

        lr = self.lrs[self.last_epoch]
        return [lr, lr * self.lr_backbone_ratio]
''')
lr_scheduler.close()

In [ ]:
tfm = open('model/transformer.py', 'w')
tfm.write('''import copy
import torch
import torch.nn as nn
import torch.nn.functional as F

from typing import Optional
from torch import nn, Tensor

def convert_hs_to_indices(hs, vocab_embed):
    # Lấy output của lớp cuối cùng và chuyển đổi kích thước
    hs_last_layer = hs[-1]  # shape: (163, 2, 256)
    hs_reshaped = hs_last_layer.transpose(0, 1)  # shape: (2, 163, 256)
    
    # Áp dụng vocab_embed
    logits = vocab_embed(hs_reshaped)  # shape: (2, 163, vocab_size)
    
    # Tìm chỉ số có xác suất cao nhất
    _, indices = torch.max(logits, dim=-1)  # shape: (2, 163)
    
    return indices
    
def mixer(previous_input, predicted_indices, sos_index, eos_index, pad_index, max_len, k):
    batch_size, seq_len = predicted_indices.shape
    
    sample_mask = torch.ones(seq_len, device=predicted_indices.device, dtype=torch.bool)
    sample_mask[0] = False
    
    # Thêm SOS token
    # next_input = torch.full((batch_size, 1), sos_index, device=predicted_indices.device)
    # formatted = torch.cat([formatted, predicted_indices], dim=1)
    # formatted = formatted[:, :max_len]
    
    next_input = torch.ones((batch_size, seq_len), device=predicted_indices.device, dtype=torch.long)
    
    #for b in range(batch_size):
    i = 0
    while(i + 10 < seq_len):
        sample_mask[i+1:i + 2 + k*4 + 1] = False
        i += 27
        
    # print(sample_mask)
    
    for i in range(seq_len):
        if sample_mask[i] == False:
            # print(i, previous_input[:, i].shape)
            next_input[:, i] = previous_input[:, i]
        else:
            next_input[:, i] = predicted_indices[:, i-1]
        # if i == 0 or i == 2:
        #     print 
            
    return next_input


class Transformer(nn.Module):

    def __init__(self, d_model, nhead, num_encoder_layers, num_decoder_layers, dim_feedforward, 
                 dropout, normalize_before, pad_token_id, num_classes, max_position_embeddings, 
                 return_intermediate_dec, num_bins, eos_index, activation="relu"):
        super(Transformer, self).__init__()
        self.embedding = DecoderEmbeddings(num_classes, d_model, pad_token_id, max_position_embeddings, dropout)
        if num_encoder_layers > 0:
            encoder_layer = TransformerEncoderLayer(d_model, nhead, dim_feedforward,
                                                    dropout, activation, normalize_before)
            encoder_norm = nn.LayerNorm(d_model) if normalize_before else None
            self.encoder = TransformerEncoder(encoder_layer, num_encoder_layers, encoder_norm)

        decoder_layer = TransformerDecoderLayer(d_model, nhead, dim_feedforward,
                                                dropout, activation, normalize_before)
        decoder_norm = nn.LayerNorm(d_model)
        self.decoder = TransformerDecoder(decoder_layer, num_decoder_layers, decoder_norm,
                                        return_intermediate=return_intermediate_dec)
        self._reset_parameters()

        self.nhead = nhead
        self.d_model = d_model
        self.num_bins = num_bins
        self.eos_index = eos_index
        self.num_encoder_layers = num_encoder_layers
        self.max_position_embeddings = max_position_embeddings

    def _reset_parameters(self):
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)

    def forward(self, src, mask, pos_embed, seq, vocab_embed):
        # flatten NxCxHxW to HWxNxC
        bs, c, h, w = src.shape
        src = src.flatten(2).permute(2, 0, 1)
        pos_embed = pos_embed.flatten(2).permute(2, 0, 1)
        mask = mask.flatten(1)
        if self.num_encoder_layers > 0:
            memory = self.encoder(src, src_key_padding_mask=mask, pos=pos_embed.half())
        else:
            memory = src

        query_embed = self.embedding.position_embeddings.weight.unsqueeze(1)
        query_embed = query_embed.repeat(1, bs, 1)
        if self.training:
            with torch.no_grad():
                for i in range(3):
                    # print(i)
                    tgt = self.embedding(seq).permute(1, 0, 2)
                    hs = self.decoder(tgt, memory, memory_key_padding_mask=mask,
                                  pos=pos_embed, query_pos=query_embed[:len(tgt)],
                                  tgt_mask=generate_square_subsequent_mask(len(tgt)).to(tgt.device))

                    predicted_indices = convert_hs_to_indices(hs, vocab_embed)
                    seq = mixer(seq, predicted_indices, sos_index=1098, eos_index=1097, pad_index=1096, max_len=hs.shape[1], k=i)
            
            tgt = self.embedding(seq).permute(1, 0, 2)
            hs = self.decoder(tgt, memory, memory_key_padding_mask=mask,
                              pos=pos_embed, query_pos=query_embed[:len(tgt)],
                              tgt_mask=generate_square_subsequent_mask(len(tgt)).to(tgt.device))

            return vocab_embed(hs[-1].transpose(0, 1))
        else:
            probs = []
            for i in range(self.max_position_embeddings):
                tgt = self.embedding(seq).permute(1, 0, 2)
                hs = self.decoder(tgt, memory, memory_key_padding_mask=mask,
                          pos=pos_embed, query_pos=query_embed[:len(tgt)],
                          tgt_mask=generate_square_subsequent_mask(len(tgt)).to(tgt.device))
                out = vocab_embed(hs.transpose(1, 2)[-1, :, -1, :])
                out = out.softmax(-1)

                # bins chars eos sos padding
                if i % 27 == 0: # coordinate or eos
                    out[:, self.num_bins:self.eos_index] = 0
                    out[:, self.eos_index+1:] = 0
                elif i % 27 == 1: # coordinate
                    out = out[:, :self.num_bins]
                else: # chars
                    out[:, :self.num_bins] = 0
                    out[:, self.eos_index:] = 0

                prob, extra_seq = out.topk(dim=-1, k=1)
                seq = torch.cat([seq, extra_seq], dim=-1)
                probs.append(prob)                
                if extra_seq[0] == self.eos_index:
                    break
            
            seq = seq[:, 1:] # remove start index
            return seq, torch.cat(probs, dim=-1)


class DecoderEmbeddings(nn.Module):
    def __init__(self, vocab_size, hidden_dim, pad_token_id, max_position_embeddings, dropout):
        super(DecoderEmbeddings, self).__init__()
        self.word_embeddings = nn.Embedding(vocab_size, hidden_dim, padding_idx=pad_token_id)
        self.position_embeddings = nn.Embedding(max_position_embeddings, hidden_dim)

        self.LayerNorm = torch.nn.LayerNorm(hidden_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        input_shape = x.size()
        seq_length = input_shape[1]
        device = x.device

        position_ids = torch.arange(seq_length, dtype=torch.long, device=device)
        position_ids = position_ids.unsqueeze(0).expand(input_shape)

        input_embeds = self.word_embeddings(x)
        position_embeds = self.position_embeddings(position_ids)

        embeddings = input_embeds + position_embeds
        embeddings = self.LayerNorm(embeddings)
        embeddings = self.dropout(embeddings)

        return embeddings


def generate_square_subsequent_mask(sz):
    r"""Generate a square mask for the sequence. The masked positions are filled with float('-inf').
        Unmasked positions are filled with float(0.0).
    """
    mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


class TransformerEncoder(nn.Module):

    def __init__(self, encoder_layer, num_layers, norm=None):
        super(TransformerEncoder, self).__init__()
        self.layers = _get_clones(encoder_layer, num_layers)
        self.num_layers = num_layers
        self.norm = norm

    def forward(self, src,
                mask: Optional[Tensor] = None,
                src_key_padding_mask: Optional[Tensor] = None,
                pos: Optional[Tensor] = None):
        output = src

        for layer in self.layers:
            output = layer(output, src_mask=mask,
                           src_key_padding_mask=src_key_padding_mask, pos=pos)

        if self.norm is not None:
            output = self.norm(output)

        return output


class TransformerDecoder(nn.Module):

    def __init__(self, decoder_layer, num_layers, norm=None, return_intermediate=False):
        super(TransformerDecoder, self).__init__()
        self.layers = _get_clones(decoder_layer, num_layers)
        self.num_layers = num_layers
        self.norm = norm
        self.return_intermediate = return_intermediate

    def forward(self, tgt, memory,
                tgt_mask: Optional[Tensor] = None,
                memory_mask: Optional[Tensor] = None,
                tgt_key_padding_mask: Optional[Tensor] = None,
                memory_key_padding_mask: Optional[Tensor] = None,
                pos: Optional[Tensor] = None,
                query_pos: Optional[Tensor] = None):
        output = tgt

        intermediate = []

        for layer in self.layers:
            output = layer(output, memory, tgt_mask=tgt_mask,
                           memory_mask=memory_mask,
                           tgt_key_padding_mask=tgt_key_padding_mask,
                           memory_key_padding_mask=memory_key_padding_mask,
                           pos=pos, query_pos=query_pos)
            if self.return_intermediate:
                intermediate.append(self.norm(output))

        if self.norm is not None:
            output = self.norm(output)
            if self.return_intermediate:
                intermediate.pop()
                intermediate.append(output)

        if self.return_intermediate:
            return torch.stack(intermediate)
            
        # print("HAHAHA")

        return output.unsqueeze(0)


class TransformerEncoderLayer(nn.Module):

    def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1,
                 activation="relu", normalize_before=False):
        super(TransformerEncoderLayer, self).__init__()
        self.self_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout)
        # Implementation of Feedforward model
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, d_model)

        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

        self.activation = _get_activation_fn(activation)
        self.normalize_before = normalize_before

    def with_pos_embed(self, tensor, pos: Optional[Tensor]):
        return tensor if pos is None else tensor + pos

    def forward_post(self,
                     src,
                     src_mask: Optional[Tensor] = None,
                     src_key_padding_mask: Optional[Tensor] = None,
                     pos: Optional[Tensor] = None):
        q = k = self.with_pos_embed(src, pos)
        src2 = self.self_attn(q, k, value=src, attn_mask=src_mask,
                              key_padding_mask=src_key_padding_mask)[0]
        src = src + self.dropout1(src2)
        src = self.norm1(src)
        src2 = self.linear2(self.dropout(self.activation(self.linear1(src))))
        src = src + self.dropout2(src2)
        src = self.norm2(src)
        return src

    def forward_pre(self, src,
                    src_mask: Optional[Tensor] = None,
                    src_key_padding_mask: Optional[Tensor] = None,
                    pos: Optional[Tensor] = None):
        src2 = self.norm1(src)
        q = k = self.with_pos_embed(src2, pos)
        src2 = self.self_attn(q, k, value=src2, attn_mask=src_mask,
                              key_padding_mask=src_key_padding_mask)[0]
        src = src + self.dropout1(src2)
        src2 = self.norm2(src)
        src2 = self.linear2(self.dropout(self.activation(self.linear1(src2))))
        src = src + self.dropout2(src2)
        return src

    def forward(self, src,
                src_mask: Optional[Tensor] = None,
                src_key_padding_mask: Optional[Tensor] = None,
                pos: Optional[Tensor] = None):
        if self.normalize_before:
            return self.forward_pre(src, src_mask, src_key_padding_mask, pos)
        return self.forward_post(src, src_mask, src_key_padding_mask, pos)


class TransformerDecoderLayer(nn.Module):

    def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1,
                 activation="relu", normalize_before=False):
        super(TransformerDecoderLayer, self).__init__()
        self.self_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout)
        self.multihead_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout)
        # Implementation of Feedforward model
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, d_model)

        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.dropout3 = nn.Dropout(dropout)

        self.activation = _get_activation_fn(activation)
        self.normalize_before = normalize_before

    def with_pos_embed(self, tensor, pos: Optional[Tensor]):
        return tensor if pos is None else tensor + pos

    def forward_post(self, tgt, memory,
                     tgt_mask: Optional[Tensor] = None,
                     memory_mask: Optional[Tensor] = None,
                     tgt_key_padding_mask: Optional[Tensor] = None,
                     memory_key_padding_mask: Optional[Tensor] = None,
                     pos: Optional[Tensor] = None,
                     query_pos: Optional[Tensor] = None):
        q = k = self.with_pos_embed(tgt, query_pos)
        tgt2 = self.self_attn(q, k, value=tgt, attn_mask=tgt_mask,
                              key_padding_mask=tgt_key_padding_mask)[0]
        tgt = tgt + self.dropout1(tgt2)
        tgt = self.norm1(tgt)
        tgt2 = self.multihead_attn(query=self.with_pos_embed(tgt, query_pos),
                                   key=self.with_pos_embed(memory, pos),
                                   value=memory, attn_mask=memory_mask,
                                   key_padding_mask=memory_key_padding_mask)[0]
        tgt = tgt + self.dropout2(tgt2)
        tgt = self.norm2(tgt)
        tgt2 = self.linear2(self.dropout(self.activation(self.linear1(tgt))))
        tgt = tgt + self.dropout3(tgt2)
        tgt = self.norm3(tgt)
        return tgt

    def forward_pre(self, tgt, memory,
                    tgt_mask: Optional[Tensor] = None,
                    memory_mask: Optional[Tensor] = None,
                    tgt_key_padding_mask: Optional[Tensor] = None,
                    memory_key_padding_mask: Optional[Tensor] = None,
                    pos: Optional[Tensor] = None,
                    query_pos: Optional[Tensor] = None):
        tgt2 = self.norm1(tgt)
        q = k = self.with_pos_embed(tgt2, query_pos)
        tgt2 = self.self_attn(q, k, value=tgt2, attn_mask=tgt_mask,
                              key_padding_mask=tgt_key_padding_mask)[0]
        tgt = tgt + self.dropout1(tgt2)
        tgt2 = self.norm2(tgt)
        tgt2 = self.multihead_attn(query=self.with_pos_embed(tgt2, query_pos),
                                   key=self.with_pos_embed(memory, pos),
                                   value=memory, attn_mask=memory_mask,
                                   key_padding_mask=memory_key_padding_mask)[0]
        tgt = tgt + self.dropout2(tgt2)
        tgt2 = self.norm3(tgt)
        tgt2 = self.linear2(self.dropout(self.activation(self.linear1(tgt2))))
        tgt = tgt + self.dropout3(tgt2)
        return tgt

    def forward(self, tgt, memory,
                tgt_mask: Optional[Tensor] = None,
                memory_mask: Optional[Tensor] = None,
                tgt_key_padding_mask: Optional[Tensor] = None,
                memory_key_padding_mask: Optional[Tensor] = None,
                pos: Optional[Tensor] = None,
                query_pos: Optional[Tensor] = None):
        if self.normalize_before:
            return self.forward_pre(tgt, memory, tgt_mask, memory_mask,
                                    tgt_key_padding_mask, memory_key_padding_mask, pos, query_pos)
        return self.forward_post(tgt, memory, tgt_mask, memory_mask,
                                 tgt_key_padding_mask, memory_key_padding_mask, pos, query_pos)


def _get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])


def build_transformer(args):
    max_position_embeddings = (2 + 25) * args.max_num_text_ins + 1
    return Transformer(
        d_model=args.tfm_hidden_dim,
        nhead=args.tfm_nheads,
        num_encoder_layers=args.tfm_enc_layers,
        num_decoder_layers=args.tfm_dec_layers,
        dim_feedforward=args.tfm_dim_feedforward,
        dropout=args.tfm_dropout,
        normalize_before=args.tfm_pre_norm,
        pad_token_id=args.padding_index,
        num_classes=args.num_classes,
        max_position_embeddings=max_position_embeddings,
        return_intermediate_dec=False,
        num_bins=args.num_bins,
        eos_index=args.eos_index,
    )


def _get_activation_fn(activation):
    """Return an activation function given a string"""
    if activation == "relu":
        return F.relu
    if activation == "gelu":
        return F.gelu
    if activation == "glu":
        return F.glu
    raise RuntimeError(F"activation should be relu/gelu, not {activation}.")
''')
tfm.close()

In [ ]:
!{PYTHON} -m torch.distributed.launch --nproc_per_node=2 main.py --data_root ./data/ --train_dataset ic15_train --output_folder ./output/icdar2015/ --resume ./ic15.pth --lr 0.00001 --epochs 371 --checkpoint_freq 5 --freeze_bn --batch_aug --tfm_pre_norm --finetune 

In [ ]:
%cd evaluation
!gdown 1ztyjczfn3YdBf6hpLuV2Vs2UJPlRdAjm
!gdown 1JxmuDsOZ-x_WO5lck2ZQZHRcjoUtUiLo
!unzip gt.zip
!unzip lexicons.zip
%cd ..

# 369

In [ ]:
!{PYTHON} main.py --eval --data_root ./data/ --val_dataset ic15_val --output_folder ./ --resume ./output/icdar2015/checkpoints/checkpoint_ep0369.pth --tfm_pre_norm --visualize

In [ ]:
!{PYTHON} evaluation/eval.py --result_path results/ep369/ic15_val.json --with_lexicon --lexicon_type 0 # used for ICDAR2013 and ICDAR2015. 0: Generic; 1: Weak; 2: Strong.

In [ ]:
!{PYTHON} evaluation/eval.py --result_path results/ep369/ic15_val.json --with_lexicon --lexicon_type 1 # used for ICDAR2013 and ICDAR2015. 0: Generic; 1: Weak; 2: Strong.

In [ ]:
!{PYTHON} evaluation/eval.py --result_path results/ep369/ic15_val.json --with_lexicon --lexicon_type 2 # used for ICDAR2013 and ICDAR2015. 0: Generic; 1: Weak; 2: Strong.

# 364

In [ ]:
!{PYTHON} main.py --eval --data_root ./data/ --val_dataset ic15_val --output_folder ./ --resume ./output/icdar2015/checkpoints/checkpoint_ep0364.pth --tfm_pre_norm --visualize

In [ ]:
!{PYTHON} evaluation/eval.py --result_path results/ep364/ic15_val.json --with_lexicon --lexicon_type 0 # used for ICDAR2013 and ICDAR2015. 0: Generic; 1: Weak; 2: Strong.

In [ ]:
!{PYTHON} evaluation/eval.py --result_path results/ep364/ic15_val.json --with_lexicon --lexicon_type 1 # used for ICDAR2013 and ICDAR2015. 0: Generic; 1: Weak; 2: Strong.

In [ ]:
!{PYTHON} evaluation/eval.py --result_path results/ep364/ic15_val.json --with_lexicon --lexicon_type 2 # used for ICDAR2013 and ICDAR2015. 0: Generic; 1: Weak; 2: Strong.

# 359

In [ ]:
!{PYTHON} main.py --eval --data_root ./data/ --val_dataset ic15_val --output_folder ./ --resume ./output/icdar2015/checkpoints/checkpoint_ep0359.pth --tfm_pre_norm --visualize

In [ ]:
!{PYTHON} evaluation/eval.py --result_path results/ep359/ic15_val.json --with_lexicon --lexicon_type 0 # used for ICDAR2013 and ICDAR2015. 0: Generic; 1: Weak; 2: Strong.

In [ ]:
!{PYTHON} evaluation/eval.py --result_path results/ep359/ic15_val.json --with_lexicon --lexicon_type 1 # used for ICDAR2013 and ICDAR2015. 0: Generic; 1: Weak; 2: Strong.

In [ ]:
!{PYTHON} evaluation/eval.py --result_path results/ep359/ic15_val.json --with_lexicon --lexicon_type 2 # used for ICDAR2013 and ICDAR2015. 0: Generic; 1: Weak; 2: Strong.

# 354

In [ ]:
!{PYTHON} main.py --eval --data_root ./data/ --val_dataset ic15_val --output_folder ./ --resume ./output/icdar2015/checkpoints/checkpoint_ep0354.pth --tfm_pre_norm --visualize

In [ ]:
!{PYTHON} evaluation/eval.py --result_path results/ep354/ic15_val.json --with_lexicon --lexicon_type 0 # used for ICDAR2013 and ICDAR2015. 0: Generic; 1: Weak; 2: Strong.

In [ ]:
!{PYTHON} evaluation/eval.py --result_path results/ep354/ic15_val.json --with_lexicon --lexicon_type 1 # used for ICDAR2013 and ICDAR2015. 0: Generic; 1: Weak; 2: Strong.

In [ ]:
!{PYTHON} evaluation/eval.py --result_path results/ep354/ic15_val.json --with_lexicon --lexicon_type 2 # used for ICDAR2013 and ICDAR2015. 0: Generic; 1: Weak; 2: Strong.

# None

In [ ]:
!{PYTHON} evaluation/eval.py --result_path results/ep369/ic15_val.json # --with_lexicon --lexicon_type 2 # used for ICDAR2013 and ICDAR2015. 0: Generic; 1: Weak; 2: Strong.

In [ ]:
!{PYTHON} evaluation/eval.py --result_path results/ep364/ic15_val.json # --with_lexicon --lexicon_type 2 # used for ICDAR2013 and ICDAR2015. 0: Generic; 1: Weak; 2: Strong.

In [ ]:
!{PYTHON} evaluation/eval.py --result_path results/ep359/ic15_val.json # --with_lexicon --lexicon_type 2 # used for ICDAR2013 and ICDAR2015. 0: Generic; 1: Weak; 2: Strong.

In [ ]:
!{PYTHON} evaluation/eval.py --result_path results/ep354/ic15_val.json # --with_lexicon --lexicon_type 2 # used for ICDAR2013 and ICDAR2015. 0: Generic; 1: Weak; 2: Strong.